In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

import time
from tqdm.notebook import trange, tqdm

In [2]:
def GetPageLinks(page_number):
    '''
    
    Запрашивает данные по странице с прощедшими играми.
   
    Возвращает quiz_link: объект list, состоящий из 12 линков формата "/game-page?id=*"
    
    '''
    
    
    # страница прошедших игр
    page_link = 'https://quizplease.ru/schedule-past?QpGameSearch%5BcityId%5D=17&QpGameSearch%5Bmonth%5D=0&QpGameSearch%5Btype%5D=1&QpGameSearch%5Bbars%5D=all&page={}&per-page=12'.format(page_number)
    
    # запрашиваем данные со страницы
    response = get(page_link)
    
    # проверяем есть ли ответ
    if not response.ok:
        return []
    
    # получаем содержимое страницы
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    # получаем по href список с линками 12 игр
    quiz_links = soup.findAll('a', {'class': 'button button-transwhite button-left button-small-line w-button'})
    quiz_links = [item.get('href') for item in quiz_links]
    
    
    return quiz_links

In [3]:
def GetSoup(link):
    
    '''
    Запрашивает информацию по линку.
    
    Возвращает soup: объект bs4.BeautifulSoup, представление текущей страницы
    
    Возвращает exam: объект bool, является игра стримом или нет
    
    '''
    
    # запрашиваем данные со страницы
    response = get('https://quizplease.ru{}'.format(link))
#     if not response.ok:
#         return []
    # получаем содержимое страницы
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # находим название игры
    game_title_number = soup.findAll('h1', {'class': 'h1 h1-left'})
    
    title = '' if not game_title_number else game_title_number[0].text
    
    # проверяем стрим это, или нет
    if 'стрим' in title:
        exam = True
    else:
        exam = False
    
    return soup, exam

In [4]:
def GetGameInformation(soup):
    
    '''
    Получает основную информацию об игре
    
    Возвращает основные характеристики объекты list
    '''
    
    # получаем название и номер игры
    game_title_number = soup.findAll('h1', {'class': 'h1 h1-left'})
    
    title = '' if not game_title_number else game_title_number[0].text
    game_number = '' if not game_title_number else game_title_number[1].text    
    
    # получаем список по классу, из которого далее будем получать информацию
    total_information = soup.findAll('div', {'class': 'game-info-column'})
    
    # находим название заведения и его адрес
    bar_name_adress = total_information[1]
    bar_name = '' if not bar_name_adress else bar_name_adress.find('div', {'class': 'text'}).text
    bar_adress = '' if not bar_name_adress else bar_name_adress.find('div', {'class': 'text text-grey'}).text
    
    # получаем дату
    date = total_information[2]

    date = (
        '' if not date else 
        date.find('div', {'class': 'text'}).text +
        ", " +
        date.find('div', {'class': 'text text-grey'}).text
    )
    
    return title, game_number, bar_name, bar_adress, date

In [5]:
def GetGameTable(link):
    
    '''
    Получаем таблицу с результатами игры
    
    Возвращает объект pandas.core.frame.DataFrame
    
    '''
    
    table = pd.read_html('https://quizplease.ru{}'.format(link), encoding='UTF-8')[0]
    
    return table

In [6]:
def GetFinalTable(table, title, game_number, bar_name, bar_adress, date, link):
    
    '''
    Формирует конечную таблицу
    
    Возвращает объект pandas.core.frame.DataFrame
    '''
    
    table['Название игры'] = title
    table['Номер игры'] = game_number
    table['Название площадки'] = bar_name
    table['Адрес площадки'] = bar_adress
    table['Дата проведения'] = date
    table['Ссылка на сайт'] = 'https://quizplease.ru' + link
    
    return table

In [1]:
final_table = pd.DataFrame()
no_inf = []
stream_count = 0

for page_number in tqdm(range(1, 170), desc='Страница'):
    quiz_links = GetPageLinks(page_number)
    for link in tqdm(quiz_links, desc='Игры'):
        error = 0
        for i in range(2):
            try:
                soup, exam = GetSoup(link)
                if not exam:
                    title, game_number, bar_name, bar_adress, date = GetGameInformation(soup)
                    table = GetGameTable(link)
                    table = GetFinalTable(table, title, game_number, bar_name, bar_adress, date, link)
                    final_table = pd.concat([final_table, table], ignore_index=True)

                else:
                    stream_count += 1
                    print('Стримов: {}'.format(stream_count))
                break
            except:
                print('Нет данных {}'.format(link))
                error += 1
                continue
        
        #print(type(no_inf))
        if error == 2:
            no_inf.append(link)
        time.sleep(0.2)

In [112]:
final_table.to_csv('qp_data_result.csv', index=True)